Imports

In [35]:
from tensorflow import keras
import tensorflow as tf
from keras import layers
from keras.datasets.cifar import load_batch
from keras import backend
import numpy as np
import os


#Dataset Load

In [36]:
path = "datasets"
num_train_samples = 50000
x_train = np.empty((num_train_samples, 3, 32, 32), dtype="uint8")
y_train = np.empty((num_train_samples,), dtype="uint8")
for i in range(1, 6):
    fpath = os.path.join(path, "data_batch_" + str(i))
    (x_train[(i - 1) * 10000 : i * 10000, :, :, :],y_train[(i - 1) * 10000 : i * 10000],) = load_batch(fpath)
fpath = os.path.join(path, "test_batch")
x_test, y_test = load_batch(fpath)
y_train = np.reshape(y_train, (len(y_train), 1))
y_test = np.reshape(y_test, (len(y_test), 1))
if backend.image_data_format() == "channels_last":
    x_train = x_train.transpose(0, 2, 3, 1)
    x_test = x_test.transpose(0, 2, 3, 1)
x_test = x_test.astype(x_train.dtype)
y_test = y_test.astype(y_train.dtype)
print(x_train.shape,x_test.shape)
# (x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()


(50000, 32, 32, 3) (10000, 32, 32, 3)


In [37]:
num_classes = 10
img_rows, img_cols = 32, 32

Normalise

In [38]:
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

In [39]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

Model definition

In [40]:
model = keras.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation="relu", input_shape=(img_rows, img_cols, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation="relu"))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation="relu"))
model.add(tf.keras.layers.Dropout(0.3))
model.add(layers.Flatten())
model.add(tf.keras.layers.Dropout(0.3))
model.add(layers.Dense(64, activation="relu"))
model.add(layers.Dense(num_classes, activation="softmax"))


model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [41]:
model.fit(x_train, y_train, epochs=15, batch_size=64)

Epoch 1/15
782/782 [==============================] - 22s 27ms/step - loss: 1.6539 - accuracy: 0.3886
Epoch 2/15
782/782 [==============================] - 20s 26ms/step - loss: 1.3087 - accuracy: 0.5297
Epoch 3/15
782/782 [==============================] - 23s 29ms/step - loss: 1.1851 - accuracy: 0.5774
Epoch 4/15
782/782 [==============================] - 22s 28ms/step - loss: 1.1016 - accuracy: 0.6090
Epoch 5/15
782/782 [==============================] - 21s 27ms/step - loss: 1.0470 - accuracy: 0.6280
Epoch 6/15
782/782 [==============================] - 21s 27ms/step - loss: 0.9987 - accuracy: 0.6447
Epoch 7/15
782/782 [==============================] - 21s 27ms/step - loss: 0.9552 - accuracy: 0.6621
Epoch 8/15
782/782 [==============================] - 21s 27ms/step - loss: 0.9197 - accuracy: 0.6764
Epoch 9/15
782/782 [==============================] - 23s 30ms/step - loss: 0.8920 - accuracy: 0.6842
Epoch 10/15
782/782 [==============================] - 24s 31ms/step - loss: 0.864

# evaluate the model

In [42]:
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print("Test accuracy:", test_acc)

313/313 - 2s - loss: 0.7606 - accuracy: 0.7370 - 2s/epoch - 6ms/step
Test accuracy: 0.7369999885559082


In [43]:
model_predictions = model.predict(x_test)

# Compute the mean of the validation set labels
mean_val_labels = np.mean(y_test)

# Compute the total variance of the validation set
total_variance = np.mean(np.square(y_test - mean_val_labels))

# Compute the bias squared
bias_squared = np.square(np.mean(y_test - model_predictions))

# Compute the variance of the model predictions
variance = total_variance - bias_squared

print("Bias squared:", bias_squared)
print("Variance:", variance)

313/313 [==============================] - 2s 6ms/step
Bias squared: 8.3709935e-21
Variance: 0.09


Saving the Keras model

In [44]:
model.save('saved_model/Classification')




INFO:tensorflow:Assets written to: saved_model/Classification\assets


INFO:tensorflow:Assets written to: saved_model/Classification\assets


converting and saving the Lite version of the Model  

In [45]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open('saved_model/LiteModel.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\zizou\AppData\Local\Temp\tmpvrv6bnai\assets


INFO:tensorflow:Assets written to: C:\Users\zizou\AppData\Local\Temp\tmpvrv6bnai\assets
